In [15]:
import functions_framework
import pandas as pd
import requests  
import sqlalchemy

@functions_framework.http
def test(request):
    connection_string = connection()
    get_cities_info(connection_string)
    return 'Data successfully added'

def get_cities_info(con_str):
    try:
        url = "https://api.api-ninjas.com/v1/city"
        headers = {"X-Api-Key": "API_KEY"}
        querystring = {"country": "DE", "min_population": "500000", "limit": "5"}

        # Make the request
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Raises an error for bad status codes
        cities_json = response.json()

        city_name = []
        city_latitude = []
        city_longitude = []
        country = []
        population = []

        for info in cities_json:
            # Extract information
            city_name.append(info['name'])
            city_latitude.append(info['latitude'])
            city_longitude.append(info['longitude'])
            country.append(info['country'])
            population.append(info['population'])

        cities_info = pd.DataFrame({
            'city_name': city_name,
            'city_latitude': city_latitude,
            'city_longitude': city_longitude,
            'country_code': country,
            'population': population
        })

        # Insert DataFrame into database
        engine = sqlalchemy.create_engine(con_str)
        cities_info.to_sql(name="cities_info", con=engine, if_exists='append', index=False)

    except requests.exceptions.RequestException as e:
        # Handle request errors
        print(f"Request error: {e}")
        raise
    except Exception as e:
        # Handle general errors
        print(f"Error: {e}")
        raise

def connection():
    schema = "gans"
    host = "MY_IP"
    user = "root"
    password = "MY_pass"
    port = 3306
    db = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return db
